# Figure 4

In [148]:
import os
from pathlib import Path
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import Phylo

cwd = os.getcwd()
if cwd.endswith('notebook'):
    os.chdir('..')
    cwd = os.getcwd()

from src.tree.itol_annotation import itol_heatmap_annotations

In [21]:
sns.set_palette('colorblind')
sns.set_style('whitegrid')
sns.set_context('paper', font_scale=1.8)
plt.rcParams['font.family'] = 'Helvetica'

palette = sns.color_palette().as_hex()

data_folder = Path('./data')
assert data_folder.is_dir()

figures_folder = Path('./figures')
assert figures_folder.is_dir()

In [33]:
gtdb_metadata = pd.read_csv(data_folder / 'gtdb_metadata.csv', index_col='ncbi_accession')

# Bacteriocins heatmap

Data generated with script `src/TBD`

In [22]:
bacteriocins_df = pd.read_csv(data_folder / 'bacteriocins.csv.gz')
bacteriocins_df.head()

,assembly_accession,protein_id,amp_id,amp_desc,evalue,bitscore,ncbi_accession,domain,gtdb_phylum,gtdb_class,gtdb_order,gtdb_family,gtdb_genus,gtdb_species,ncbi_organism_name
0,GCA_000007325.1,AAL94370.1,pg_hydrolase,Amidase_2+PG_binding_1,1.150000e-08,61.95,GCA_000007325.1,Bacteria,Fusobacteriota,Fusobacteriia,Fusobacteriales,Fusobacteriaceae,Fusobacterium,Fusobacterium nucleatum,Fusobacterium nucleatum subsp. nucleatum ATCC ...
1,GCA_000007325.1,AAL94673.1,pg_hydrolase,Peptidase_M23+LysM,6.500000e-08,64.00,GCA_000007325.1,Bacteria,Fusobacteriota,Fusobacteriia,Fusobacteriales,Fusobacteriaceae,Fusobacterium,Fusobacterium nucleatum,Fusobacterium nucleatum subsp. nucleatum ATCC ...
2,GCA_000008885.1,BAC24414.1,pg_hydrolase,Amidase_3+AMIN,3.900000e-15,106.70,GCA_000008885.1,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Wigglesworthia,Wigglesworthia glossinidia_A,Wigglesworthia glossinidia endosymbiont of Glo...
3,GCA_000010565.1,BAF58967.1,pg_hydrolase,Amidase_3+AMIN,6.000000e-15,121.55,GCA_000010565.1,Bacteria,Bacillota_B,Desulfotomaculia,Desulfotomaculales,Pelotomaculaceae,Pelotomaculum,Pelotomaculum thermopropionicum,Pelotomaculum thermopropionicum SI
4,GCA_000010565.1,BAF60354.1,pg_hydrolase,Amidase_2+LysM,6.000135e-15,59.45,GCA_000010565.1,Bacteria,Bacillota_B,Desulfotomaculia,Desulfotomaculales,Pelotomaculaceae,Pelotomaculum,Pelotomaculum thermopropionicum,Pelotomaculum thermopropionicum SI


### Make sure the bacteriocins dataset is congruent with the PGH dataset 

In [26]:
pgh_df = pd.read_csv(data_folder / 'pgh_proteins.csv')
pgh_ids = set(pgh_df['protein_id'].unique())
pgh_bacteriocins = bacteriocins_df[bacteriocins_df['amp_id'] == 'pg_hydrolase']
pgh_bacteriocins_ids = set(pgh_bacteriocins['protein_id'].unique())

assert len(pgh_df) == len(pgh_bacteriocins)
assert len(pgh_bacteriocins_ids - pgh_ids) == 0
assert len(pgh_ids - pgh_bacteriocins_ids) == 0

### Assign phylum group

Just like in figre 1 – GTDB phyla sub group such as Pseudomonata_A, Pseudomonata_B, etc. are merged into Pseudomonata for clarity. Unamed phyla are ignored.

In [137]:
def is_named_phylum(phylum):
    if re.match(r'^[A-Z0-9\-_]+$', phylum) is not None:
        return False

    blacklist = {
        'BMS3Abin14',
        'BS750m-G25',
        'BS750m-G34',
        'JdFR-76',
        'T1Sed10-126',
        'SpSt-1190',
        'B1Sed10-29',
        'EX4484-52',
    }
    return phylum not in blacklist


def get_phylum_group(phylum):
    m = re.match(r'^(.+)_[A-Z]$', phylum)
    if m is not None:
        return m[1]
    else:
        return phylum


named_phyla = [
    p for p in gtdb_metadata['gtdb_phylum'].unique()
    if is_named_phylum(p)
]
map_phylm_groups = {
    phylum: get_phylum_group(phylum)
    for phylum in named_phyla
}
bacteriocins_df['phylum_group'] = bacteriocins_df['gtdb_phylum'].apply(lambda p: map_phylm_groups.get(p))

In [45]:
arc_phylum_group_tree = Phylo.read(data_folder / 'figure1' / 'ar53_phylum_groups.tree', 'phyloxml')
bac_phylum_group_tree = Phylo.read(data_folder / 'figure1' / 'bac120_phylum_groups.tree', 'phyloxml')

arc_phylum_groups = sorted([l.name for l in arc_phylum_group_tree.get_terminals()])
bac_phylum_groups = sorted([l.name for l in bac_phylum_group_tree.get_terminals()])
all_phylum_groups = sorted(set(arc_phylum_groups) | set(bac_phylum_groups))

print(f'Number of archaeal phylum groups: {len(arc_phylum_groups):,}')
print(f'Number of bacterial phylum groups: {len(bac_phylum_groups):,}')

Number of archaeal phylum groups: 14
Number of bacterial phylum groups: 34


In [138]:
bacteriocins_subset = bacteriocins_df[
    bacteriocins_df['phylum_group'].isin(all_phylum_groups)
].copy()
p = 100 * len(bacteriocins_subset) / len(bacteriocins_df)

print(f'Number of bacteriocins included within phyla subset: {len(bacteriocins_subset):,} ({p:.0f}% of total)')

Number of bacteriocins included within phyla subset: 145,436 (98% of total)


## Selection of bacteriocins shared between archaea and bacteria

In [54]:
arc_bacteriocins = set(bacteriocins_subset[bacteriocins_subset['domain'] == 'Archaea']['amp_id'].unique())
bac_bacteriocins = set(bacteriocins_subset[bacteriocins_subset['domain'] == 'Bacteria']['amp_id'].unique())

shared_bacteriocins = sorted(arc_bacteriocins & bac_bacteriocins)

print(f'Number of bacteriocins shared between archaea and bacteria: {len(shared_bacteriocins):,}')

Number of bacteriocins shared between archaea and bacteria: 86


In [92]:
counts_per_amp = bacteriocins_subset[
    bacteriocins_subset['amp_id'] .isin(shared_bacteriocins)
][
    ['amp_id', 'assembly_accession'
]].groupby('amp_id').nunique().rename(columns={'assembly_accession': 'n_assemblies'})

counts_per_domain = bacteriocins_subset[
    ['domain', 'amp_id', 'assembly_accession'
]].groupby(['domain', 'amp_id']).nunique()

archaea_counts = counts_per_domain.loc['Archaea'].copy().rename(columns={'assembly_accession': 'n_archaea'})
bacteria_counts = counts_per_domain.loc['Bacteria'].copy().rename(columns={'assembly_accession': 'n_bacteria'})

def compute_clade_count(df, amp_id, domain, clade='gtdb_class'):
    return len(df[
        (df['domain'] == domain) &
        (df['amp_id'] == amp_id)
    ][clade].unique())

amp_table = pd.merge(
    counts_per_amp,
    archaea_counts,
    how='left',
    on='amp_id',
)
amp_table = pd.merge(
    amp_table,
    bacteria_counts,
    how='left',
    on='amp_id',
)
amp_table['percent_archaea'] = (
    100 * amp_table['n_archaea'] / len(gtdb_metadata[gtdb_metadata['domain'] == 'Archaea'])
).round(2)
amp_table['percent_bacteria'] = (
    100 * amp_table['n_bacteria'] / len(gtdb_metadata[gtdb_metadata['domain'] == 'Bacteria'])
).round(2)
amp_table['percent_diff'] = (amp_table['percent_bacteria'] - amp_table['percent_archaea']).round(2)
amp_table['n_archaeal_phyla'] = [
    compute_clade_count(bacteriocins_df, amp_id, domain='Archaea', clade='gtdb_phylum')
    for amp_id in amp_table.index
]
amp_table['n_bacterial_phyla'] = [
    compute_clade_count(bacteriocins_df, amp_id, domain='Bacteria', clade='gtdb_phylum')
    for amp_id in amp_table.index
]
amp_table['n_archaeal_classes'] = [
    compute_clade_count(bacteriocins_df, amp_id, domain='Archaea', clade='gtdb_class')
    for amp_id in amp_table.index
]
amp_table['n_bacterial_classes'] = [
    compute_clade_count(bacteriocins_df, amp_id, domain='Bacteria', clade='gtdb_class')
    for amp_id in amp_table.index
]

# Selection criteria: at least 2 genomes and 2 distinct phyla in each domain.
amp_table = amp_table[
    (amp_table['n_archaea'] >= 2) & 
    (amp_table['n_archaeal_phyla'] >= 2) & 
    (amp_table['n_bacteria'] >= 2) &
    (amp_table['n_bacterial_phyla'] >= 2)
].copy()

amp_table = amp_table.sort_values(['n_archaea'], ascending=False)

amp_table

,n_assemblies,n_archaea,n_bacteria,percent_archaea,percent_bacteria,percent_diff,n_archaeal_phyla,n_bacterial_phyla,n_archaeal_classes,n_bacterial_classes
amp_id,,,,,,,,,,
pg_hydrolase,41535,174,41361,4.70,81.68,76.98,13,169,23,459
Linocin_M18,2559,174,2385,4.70,4.71,0.01,9,67,21,131
FlvA2h,13,6,7,0.16,0.01,-0.15,4,6,4,7
Lactococcin_A,149,5,144,0.13,0.28,0.15,4,26,5,42
Nocardithiocin,43,5,38,0.13,0.08,-0.05,2,11,2,16
ComX4,52,4,48,0.11,0.09,-0.02,3,12,3,14
Competence,39,4,35,0.11,0.07,-0.04,4,8,4,10
Lactococcin_Q_chain_b,26,4,22,0.11,0.04,-0.07,2,13,2,15
Enterocin_NKR_5_3D,28,4,24,0.11,0.05,-0.06,4,10,4,14


Putative bacteriocins excluded from the heatmap:

- `Linocin M18` is shown to have genuine use for harvesting iron in Pyrococcus furiosus: https://www.uniprot.org/uniprotkb/Q8U1L4/entry
- `FlvA2h` and `FlvA2f` are part of a `Lantipeptide` in _Ruminococcus flavefaciens_ but these two genes are not the active part as per: https://www.uniprot.org/uniprotkb/P0DQM0/entry

Interesting but not sure if we should include:
- `Competence`, `ComX2` and `ComX4` are both `comX` in _B. subtilis_. mimic of comX can be used to mess with quorum sensing: https://doi.org/10.1016/j.pbi.2004.05.008
- Same story but less clearcut for `Auto_Inducing_Peptide_III` which map to TIGR entry TIGR04223: https://www.ncbi.nlm.nih.gov/genome/annotation_prok/evidence/TIGR04223/

Name change:
- `PaeM` is better known as `Colicin M` as per automatic Pfam annotations: https://www.uniprot.org/uniprotkb/Q88A25/entry
- `Bacteriocin_31` is Pfam domain `Bacteriocin_II`
- `Gardimycin_(actagardine)` renamed `Actagardine`

In [120]:
display_name_map = {
    'Competence': 'ComX homolog',
    'ComX2': 'ComX homolog',
    'ComX4': 'ComX homolog',
    'pg_hydrolase': 'Peptidoglycan hydrolase',
    'PaeM': 'Colicin M',
    'Bacteriocin_31': 'Bacteriocin II',
    'Gardimycin_(actagardine)': 'Actagardine',
    'Lactococcin_Q_chain_b': 'Lactococcin Q',
    'Enterocin_NKR_5_3D': 'Enterocin NKR-5-3D',
    'EnterocinP': 'Enterocin P',
    'Subtilosin_(SboX)': 'Subtilosin',
    'GarvieacinQ garQ': 'Garvieacin Q',
}
def set_display_name(name):
    if name in display_name_map:
        return display_name_map[name]
    else:
        return name.replace('_', ' ')
    
bacteriocins_df['display_name'] = bacteriocins_df['amp_id'].apply(set_display_name)

amp_id_to_diplay_name = {
    tpl.amp_id: tpl.display_name
    for tpl in bacteriocins_df.itertuples()
}

In [125]:
bacteriocins_to_exclude = {
    'pg_hydrolase',
    'Auto_Inducing_Peptide_III',
    'Linocin_M18',
    'FlvA2f',
    'FlvA2h',
    'Competence',
    'ComX2',
    'ComX4',
}
bacteriocins_shortlist = set({
    amp_id_to_diplay_name[amp_id]
    for amp_id in amp_table.index
    if amp_id not in bacteriocins_to_exclude
})
bacteriocins_shortlist

{'Actagardine',
 'Bacteriocin II',
 'Caulonodin III',
 'Colicin M',
 'Enterocin 1071B',
 'Enterocin NKR-5-3D',
 'Enterocin P',
 'Garvicin Q',
 'Lactococcin A',
 'Lactococcin MMFII',
 'Lactococcin Q',
 'Nocardithiocin',
 'Piricyclamide 7005E2',
 'Sakacin A'}

## Selection of bacteriocins present in bacteria only

Focus on 4 well-known example as illustration that not all bacteriocins are found in archaea.

In [129]:
bacteriocins_df[
    ~(bacteriocins_df['amp_id'].isin(shared_bacteriocins))
][['display_name', 'assembly_accession']].groupby('display_name').nunique().rename(columns={
    'assembly_accession': 'n_genomes'
}).sort_values('n_genomes', ascending=False).head(20)

,n_genomes
display_name,
Lactococcin 972,273
Closticin 574,221
UviB,214
Colicin E6,182
GarvieacinQ garQ,172
Colicin E9,167
Bacteriocin IIc,161
Microviridin K,93
ComC,61


In [156]:
bacteria_only_shortlist = [
    'Lactococcin 972',
    'Closticin 574',
    'Colicin E6',
    'Colicin E9',
]

In [157]:
bacteriocins_to_plot = [
    amp_id_to_diplay_name[amp_id] 
    for amp_id in amp_table.index
    if amp_id not in bacteriocins_to_exclude
] + bacteria_only_shortlist

print(f'Number of bacteriocins: {len(bacteriocins_to_plot)}')

bacteriocins_to_plot

Number of bacteriocins: 18


['Lactococcin A',
 'Nocardithiocin',
 'Lactococcin Q',
 'Enterocin NKR-5-3D',
 'Lactococcin MMFII',
 'Sakacin A',
 'Enterocin P',
 'Garvicin Q',
 'Bacteriocin II',
 'Enterocin 1071B',
 'Colicin M',
 'Piricyclamide 7005E2',
 'Caulonodin III',
 'Actagardine',
 'Lactococcin 972',
 'Closticin 574',
 'Colicin E6',
 'Colicin E9']

## Make heatmap

In [158]:
def compute_heatmap_data(
    gtdb_metadata, 
    bacteriocins_df, 
    phylum_groups, 
    bacteriocin_names, 
):
    gtdb_metadata['phylum_group'] = gtdb_metadata['gtdb_phylum'].apply(lambda p: map_phylm_groups.get(p))
    metadata = gtdb_metadata[gtdb_metadata['phylum_group'].isin(phylum_groups)].reset_index()
    phylum_count_df = metadata[
        ['phylum_group', 'ncbi_accession']
    ].groupby('phylum_group').count().rename(columns={
        'ncbi_accession': 'n_genomes'
    })
    phylum_count = {
        phylum: phylum_count_df.loc[phylum, 'n_genomes']
        for phylum in phylum_groups
    }

    heatmap_data = {
        'phylum_group': [],
        'n_assemblies': [],
    }
    for bacteriocin in bacteriocin_names:
        heatmap_data[bacteriocin] = []

    for phylum in phylum_groups:
        bacteriocin_count = bacteriocins_df[
            bacteriocins_df['display_name'].isin(bacteriocin_names) &
            (bacteriocins_df['phylum_group'] == phylum)
        ][['display_name', 'ncbi_accession']].groupby('display_name').nunique().rename(columns={
            'ncbi_accession': 'n_genomes'
        })

        heatmap_data['phylum_group'].append(phylum)
        heatmap_data['n_assemblies'].append(phylum_count[phylum])
        for bacteriocin in bacteriocin_names:
            if bacteriocin in bacteriocin_count.index:
                count = bacteriocin_count.loc[bacteriocin, 'n_genomes']
            else:
                count = 0
            
            heatmap_data[bacteriocin].append(count)

    return pd.DataFrame.from_dict(heatmap_data).set_index('phylum_group', drop=True)

In [159]:
archaeal_heatmap_df = compute_heatmap_data(
    gtdb_metadata, 
    bacteriocins_df[bacteriocins_df['domain'] == 'Archaea'].copy(), 
    phylum_groups=arc_phylum_groups, 
    bacteriocin_names=bacteriocins_to_plot, 
)
archaeal_heatmap_df

,n_assemblies,Lactococcin A,Nocardithiocin,Lactococcin Q,Enterocin NKR-5-3D,Lactococcin MMFII,Sakacin A,Enterocin P,Garvicin Q,Bacteriocin II,Enterocin 1071B,Colicin M,Piricyclamide 7005E2,Caulonodin III,Actagardine,Lactococcin 972,Closticin 574,Colicin E6,Colicin E9
phylum_group,,,,,,,,,,,,,,,,,,,
Aenigmatarchaeota,132,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
Altiarchaeota,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Asgardarchaeota,188,2,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
Hadarchaeota,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Halobacteriota,739,0,4,1,0,1,1,2,0,1,1,0,0,1,1,0,0,0,0
Hydrothermarchaeota,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Iainarchaeota,68,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Methanobacteriota,207,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Micrarchaeota,242,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0


In [160]:
bacterial_heatmap_df = compute_heatmap_data(
    gtdb_metadata, 
    bacteriocins_df[bacteriocins_df['domain'] == 'Bacteria'].copy(), 
    phylum_groups=bac_phylum_groups, 
    bacteriocin_names=bacteriocins_to_plot, 
)
bacterial_heatmap_df.head()

,n_assemblies,Lactococcin A,Nocardithiocin,Lactococcin Q,Enterocin NKR-5-3D,Lactococcin MMFII,Sakacin A,Enterocin P,Garvicin Q,Bacteriocin II,Enterocin 1071B,Colicin M,Piricyclamide 7005E2,Caulonodin III,Actagardine,Lactococcin 972,Closticin 574,Colicin E6,Colicin E9
phylum_group,,,,,,,,,,,,,,,,,,,
Acidobacteriota,1094,4,2,0,0,0,1,1,2,0,0,2,4,0,0,0,0,1,2
Actinomycetota,4027,11,8,5,2,0,0,2,0,0,2,4,6,2,7,139,143,17,11
Aquificota,84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Armatimonadota,187,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
Bacillota,10923,11,3,7,12,2,1,8,3,15,17,3,0,2,0,131,60,10,17


In [161]:
def make_heatmap_list(heatmap_df, amp_ids):
    heatmap_data = []
    for phylum in heatmap_df.index:
        row = [phylum]
        heatmap_row = heatmap_df.loc[phylum]
        for amp_id in amp_ids:
            c = round(100 * heatmap_row[amp_id] / heatmap_row['n_assemblies'], 3)
            row.append(c)

        heatmap_data.append(row)

    return heatmap_data

In [162]:
archaeal_heatmap_data = make_heatmap_list(archaeal_heatmap_df, bacteriocins_to_plot)

itol_heatmap_annotations(
        archaeal_heatmap_data,
        field_labels=bacteriocins_to_plot,
        output_path=data_folder / 'figure4' / 'ar53_phylum_subset.heatmap.txt',
        dataset_label='% genomes in archaeal phylum',
        color_min='#ffffff',
        color_max='#f58231',
        min_value=0,
        max_value=2,
    )

In [163]:
bacterial_heatmap_data = make_heatmap_list(bacterial_heatmap_df, bacteriocins_to_plot)

itol_heatmap_annotations(
        bacterial_heatmap_data,
        field_labels=bacteriocins_to_plot,
        output_path=data_folder / 'figure4' / 'bac120_phylum_subset.heatmap.txt',
        dataset_label='% genomes in bacterial phylum',
        color_min='#ffffff',
        color_max='#0173b2',
        min_value=0,
        max_value=4,
    )